In [35]:
import pandas as pd
import numpy as np

In [39]:
all = pd.read_csv('Dynasty_Ranking_23_24_All.csv')
qb = pd.read_csv('Dynasty_Ranking_23_24_QB.csv')
rb = pd.read_csv('Dynasty_Ranking_23_24_RB.csv')
wr = pd.read_csv('Dynasty_Ranking_23_24_WR.csv')
te = pd.read_csv('Dynasty_Ranking_23_24_TE.csv') 
rook = pd.read_csv('Dynasty_Ranking_23_24_Rookie.csv') 

In [40]:
# DATA PREP

# select columns wanted
all = all[["Rank", "Name", "Team", "Age"]]

# QB
qb = qb[["Rank", "Name"]]
qb = qb.assign(Pos = 'QB')
qb['Rank'] = qb['Rank'].astype(str)
qb['Pos_Rnk'] = qb['Pos'] + "-" + qb['Rank']

# RB
rb = rb[["Rank", "Name"]]
rb = rb.assign(Pos = 'RB')
rb['Rank'] = rb['Rank'].astype(str)
rb['Pos_Rnk'] = rb['Pos'] + "-" + rb['Rank']

# WR
wr = wr[["Rank", "Name"]]
wr = wr.assign(Pos = 'WR')
wr['Rank'] = wr['Rank'].astype(str)
wr['Pos_Rnk'] = wr['Pos'] + "-" + wr['Rank']

# TE
te = te[["Rank", "Name"]]
te = te.assign(Pos = 'TE')
te['Rank'] = te['Rank'].astype(str)
te['Pos_Rnk'] = te['Pos'] + "-" + te['Rank']

# Rookie
rook = rook[["Rank", "Name"]]
rook = rook.rename(columns = {'Rank': 'Rook_Rnk'})
rook['Rook_Rnk'] = rook['Rook_Rnk'].astype(str)

# Join data
comb = pd.concat([qb, rb, wr, te])
comb2 = pd.merge(comb, rook, on = 'Name', how = 'left')
all_fnl = pd.merge(all, comb2, on = 'Name', how = 'left')

# Select cols and final clean
all_fnl = all_fnl[["Rank_x", "Name", "Team", "Age", "Pos", "Pos_Rnk", "Rook_Rnk"]]
all_fnl = all_fnl.rename(columns = {'Rank_x': 'Rank'})
all_fnl = all_fnl.replace(np.nan, '', regex=True)

In [42]:
# Write csv
all_fnl.to_csv('Dynasty_23_24_Clean.csv', index = False)